# Модель VAR (пакет [`statsmodels`](https://www.statsmodels.org/stable/index.html))

In [ ]:
import numpy as np
import pandas as pd

from statsmodels.tsa.api import VAR 

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

In [ ]:
rates = web.DataReader(name=['TB3MS', 'GS10', 'AAA', 'BAA'], data_source='fred', start='1995-01-01')

y = rates.diff().dropna()

y.plot(subplots=True)
plt.show()

## Подгонка VAR заданного порядка

In [ ]:
mod = VAR(y, missing='drop')

res = mod.fit(maxlags=2, ic=None, trend='c')

res.summary()

## Подгонка VAR "оптимального" порядка

In [ ]:
mod = VAR(y, missing='drop')

res = mod.fit(maxlags=2, ic='bic', trend='c')

res.k_ar

## Визуализация для подогнанной модели

График остатков

In [ ]:
res.resid.plot(subplots=True)

plt.show()

In [ ]:
# ACF для остатков
res.plot_acorr()
plt.show()

## Диагностика модели

In [ ]:
# тест на серийную корреляцию
res.test_whiteness(nlags=10, signif=0.05, adjusted=True).summary()

## Прогнозирование

In [ ]:
# преобразуем y в формат numpy
res.forecast(y=y.tail(res.k_ar).values, steps=10)

Визуализация численного прогноза

In [ ]:
fig = res.plot_forecast(steps=20, plot_stderr=False)

# изменим положение легенды для каждого подграфика
for i in range(y.shape[1]):
	fig.axes[i].legend(loc=3)

plt.show()

Визуализация численного прогноза с доверительным интервалом

In [ ]:
fig = res.plot_forecast(steps=20, plot_stderr=True)

# изменим положение легенды для каждого подграфика
for i in range(y.shape[1]):
	fig.axes[i].legend(loc=3)

plt.show()

## IRF

In [ ]:
res.irf(periods=10).plot(signif=0.05)
plt.show()

## FEVD

In [ ]:
res.fevd(periods=10).plot()

plt.show()

In [ ]:
# res.fevd(periods=10).summary()

## Причинность

In [ ]:
res.test_causality(caused=['AAA', 'BAA'], causing=['TB3MS', 'GS10'], signif=0.01).summary()

In [ ]:
test_causality(caused= ['TB3MS', 'GS10'], causing=['AAA', 'BAA'], signif=0.01).summary()